In [1]:
# !git clone https://github.com/sberbank-ai/sber-swap.git

Cloning into 'sber-swap'...
remote: Enumerating objects: 704, done.
remote: Total 704 (delta 0), reused 0 (delta 0), pack-reused 704
Receiving objects: 100% (704/704), 94.00 MiB | 20.26 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [1]:
# cd sber-swap/
# 이 위치에서 실행

/home/j-k8e104/FaceSwap2/sber-swap


In [2]:
# !source venv/bin/activate

In [3]:
import yaml,os, subprocess, io, zipfile, torch, cv2
from flask import Flask, escape, request, send_file, jsonify
from werkzeug.utils import secure_filename
from pathlib import Path as PPath
import torch, requests, random, cv2
import numpy as np
from PIL import Image, ImageOps
from animegan.inference import Transformer
from transformers import AutoImageProcessor, Swin2SRForImageSuperResolution
import matplotlib.pyplot as plt
from animegan.utils import read_image, resize_image
from omegaconf import OmegaConf

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

anime_transformer = Transformer('hayao')
processor = AutoImageProcessor.from_pretrained("caidas/swin2SR-classical-sr-x2-64",device="cuda")
model = Swin2SRForImageSuperResolution.from_pretrained("caidas/swin2SR-classical-sr-x2-64").cuda()

/tmp/ipykernel_1279928/3407991928.py:2: DeprecationWarning: 'flask.escape' is deprecated and will be removed in Flask 2.4. Import 'markupsafe.escape' instead.
  from flask import Flask, escape, request, send_file, jsonify


Weight loaded, ready to predict


In [4]:
app = Flask(__name__)

def DeleteAllFiles(directory):
    for tempfile in os.scandir(directory):
        if tempfile.is_file():
            os.remove(tempfile.path)
    
@app.route('/', methods=['POST','GET'])
def hello():
    name = request.args.get("name", "World")
    return f'Hello, {escape(name)}!'

In [5]:
@app.route('/flask2/faceswap', methods=['POST'])
def faceswapImage():
    if request.method == 'POST':
        DeleteAllFiles('./examples/tempfiles/')
        # Check if the request contains an image
        if 'image_1' not in request.files:
            return "No image file found in the request", 400

        # Save the uploaded image
        source_image = request.files['image_1']
        if source_image.mode == 'RGBA':
            source_image = source_image.convert('RGB')
        source_filename = secure_filename(source_image.filename)
        source_image_path = os.path.join('examples/tempfiles', source_filename)
        source_image.save(source_image_path)
        target_image = request.files['image_2']
        if target_image.mode == 'RGBA':
            target_image = target_image.convert('RGB')
        target_filename = secure_filename(target_image.filename)
        target_image_path = os.path.join('examples/tempfiles', target_filename)
        target_image.save(target_image_path)
        output_image_name = "good.png"
        instruction = f"python inference.py --source_paths {source_image_path} --target_image {target_image_path} --out_image_name examples/tempfiles/{output_image_name} --image_to_image True"
#         os.system(instruction)
        subprocess.run(instruction, shell=True)
        # Return the saved image as a response
        if (output_image_name.split(".")[-1] == "png"):
            return send_file(PPath(f'examples/tempfiles/{output_image_name}'), mimetype='image/png')
        return send_file(PPath(f'examples/tempfiles/{output_image_name}'), mimetype='image/jpeg')

In [6]:
@app.route('/flask2/scaleup', methods=['POST'])
def scaleupImage():
    if 'image_1' not in request.files:
        return 'No file part', 400

    image_file = request.files['image_1']

    # Check if the file is empty
    if image_file.filename == '':
        return 'No selected file', 400

    # Open the uploaded file as a PIL Image
    image = Image.open(image_file)
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    #image = Image.open(requests.get(url, stream=True).raw)
    
    # prepare image for the model
    inputs = processor(image, return_tensors="pt")
    inputs.to("cuda")
    # forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.moveaxis(output, source=0, destination=-1)
    output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
    # you can visualize `output` with `Image.fromarray`
    img = Image.fromarray(output)
    img.save("examples/tempfiles/sr_result.png")
    # Send the image as a response with the appropriate content type
    return send_file(PPath(f'examples/tempfiles/sr_result.png'), mimetype='image/png')

In [7]:
@app.route('/flask2/animestyle', methods=['POST'])
def animestyle():
    if 'image_1' not in request.files:
        return 'No file part', 400

    image_file = request.files['image_1']

    # Check if the file is empty
    if image_file.filename == '':
        return 'No selected file', 400

    # Open the uploaded file as a PIL Image
    image = Image.open(image_file)
    image = ImageOps.exif_transpose(image)
    
    if image.mode == 'RGBA':
        image = image.convert('RGB')
    imgArray = np.asarray(image)
    anime_img = (anime_transformer.transform(imgArray) + 1) / 2
    anime_img = anime_img[0]
    anime_img = (anime_img * 255).astype(np.uint8)
    output = Image.fromarray(anime_img)
    output.save("examples/tempfiles/as_result.png")
    # Send the image as a response with the appropriate content type
    return send_file(PPath(f'examples/tempfiles/as_result.png'), mimetype='image/png')

In [ ]:
app.run(host="0.0.0.0", port=5001)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://70.12.130.121:5001
Press CTRL+C to quit
[2023-05-18 23:57:42,359] ERROR in app: Exception on /flask2/animestyle [POST]
Traceback (most recent call last):
  File "/home/j-k8e104/.local/lib/python3.9/site-packages/flask/app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/j-k8e104/.local/lib/python3.9/site-packages/flask/app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/j-k8e104/.local/lib/python3.9/site-packages/flask/app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/j-k8e104/.local/lib/python3.9/site-packages/flask/app.py", line 1469, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/tmp/ipykernel_1279928/2070023512.py", line 19, in animestyle
    anime_img = (anime_transformer.transform(imgArray) + 1) / 2
  File

input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
loading ./coordinate_reg/model/2d106det 0
input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
List of source paths:  ['examples/tempfiles/sent_image1.png']
List of target paths:  []


100%|██████████| 1/1 [00:00<00:00, 75.75it/s]
1it [00:00, 360.21it/s]
1it [00:00, 4744.69it/s]
100%|██████████| 1/1 [00:00<00:00, 25731.93it/s]


Swapped Image saved with path examples/tempfiles/good.png
Total time:  1.808093786239624


...

13.124.65.156 - - [19/May/2023 00:39:01] "POST /flask2/faceswap HTTP/1.1" 200 -
[00:39:47] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.5.0. Attempting to upgrade...
[00:39:47] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
[00:39:47] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.


input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
loading ./coordinate_reg/model/2d106det 0
input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
List of source paths:  ['examples/tempfiles/sent_image1.png']
List of target paths:  []


100%|██████████| 1/1 [00:00<00:00, 80.28it/s]
1it [00:00, 318.16it/s]
1it [00:00, 4583.94it/s]
100%|██████████| 1/1 [00:00<00:00, 29127.11it/s]


Swapped Image saved with path examples/tempfiles/good.png
Total time:  1.18515944480896


13.124.65.156 - - [19/May/2023 00:39:50] "POST /flask2/faceswap HTTP/1.1" 200 -
13.124.65.156 - - [19/May/2023 01:19:18] "POST /flask2/faceswap HTTP/1.1" 400 -
